In [84]:
import pandas as pd
import sys
sys.path.append("..")
from src.Utils import get_ents_spans

data = pd.read_parquet("../data/dataset/shs100k2/data.parquet").reset_index()


In [139]:
data_spans = pd.DataFrame(pd.json_normalize(data.IOB.apply(get_ents_spans)))
data["WoA"] = data_spans.WoA
data.WoA = data.WoA.apply(lambda x: x if x == x else [])
data["Artist"] = data_spans.Artist
data.Artist = data.Artist.apply(lambda x: x if x == x else [])

data["WoA_count"] = data.WoA.apply(len)
data["Artist_count"] = data.Artist.apply(len)
data["WoA_missing"] = data.WoA_count == 0
data["Artist_missing"] = data.Artist_count == 0
data["WoA_first"] = data.WoA.apply(lambda x: x[0] if len(x) > 0 else None)
data["Artist_first"] = data.Artist.apply(lambda x: x[0] if len(x) > 0 else None)
data["WoA_len"] = data.WoA_first.apply(lambda x: x[1] + 1 - x[0] if isinstance(x, tuple) else None)
data["Artist_len"] = data.Artist_first.apply(lambda x: x[1] + 1 - x[0] if isinstance(x, tuple) else None)
data["WoA_start"] = data.WoA_first.apply(lambda x: x[0] if x else None)
data["Artist_start"] = data.Artist_first.apply(lambda x: x[0] if x else None)


In [140]:
data[
    ["split", "set_id", "yt_id", "WoA_count", "Artist_count", 
     "WoA_len", "Artist_len", "WoA_missing", "Artist_missing",
     "WoA_start", "Artist_start"]
    ].groupby("split").agg(
        {"set_id": pd.Series.nunique, "yt_id": "count", 
         "WoA_count": ["min", "max"], "Artist_count": ["min", "max"],
         "WoA_missing": "sum", "Artist_missing": "sum",
         "WoA_len": ["median", "max"], "Artist_len": ["median", "max"],
         "WoA_start": "median", "Artist_start": "median"
         }
    )


set_id  yt_id WoA_count     Artist_count     WoA_missing  \
      nunique  count       min max          min max         sum   
split                                                             
TEST     1655   7755         0   3            0   6           7   
TRAIN    5304  62486         0   4            0   5           4   
VAL      1811   7924         0   3            0   4           6   

      Artist_missing WoA_len       Artist_len       WoA_start Artist_start  
                 sum  median   max     median   max    median       median  
split                                                                       
TEST              28     3.0  15.0        2.0  10.0       3.0          0.0  
TRAIN             31     3.0  19.0        2.0   9.0       3.0          0.0  
VAL               21     3.0  16.0        2.0   8.0       3.0          0.0